# Librarys

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

**Project**

A report by --- speaks to the importans of public libraries for ----. In this project we provide simple statistics and comparisons of how librabries in munusipalities relate to other (----). 
This project is in no way ment to claim a causal conection, as we are simple investigating correlations. 


Report: https://www.roskildebib.dk/sites/roskilde.ddbcms.dk/files/files/news/roskildebib_folkebibliotekets_betydning_for_borgerne_i_danmark_eng_final_0.pdf


Imports and set magics:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

import datetime

import pandas_datareader # install with `pip install pandas-datareader`
import pydst # install with `pip install git+https://github.com/elben10/pydst`


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})



# Read and clean data

# Importing dataset 1: Libraries by manusipalities 

**Importing data**

Import your data, either through an API or manually, and load it. 

In [ ]:
#Object for interacting with DST 
Dst = pydst.Dst(lang='en') # setup data loader with the langauge 'english'

In [ ]:
#Subjects within DST 
Dst.get_subjects()

In [ ]:
#List of tables within the subject with index 2
tables = Dst.get_tables(subjects=['7'])
display(tables)

In [ ]:
#List of variables from the tabel, AKU110K from subject 2
tables[tables.id =='BIB1']

In [ ]:
#list of variables (again?)
bib_var = Dst.get_variables(table_id = 'BIB1')
bib_var

In [ ]:
#Values of the variables 
bib_vars  = Dst.get_variables(table_id = 'BIB1')

for id in [ 'BNØGLE', 'Tid']:
    print(id)
    values = bib_vars.loc[bib_vars.id ==id, ['values']].values[0,0]
    for value in values:
        print(f'id = {value["id"]}, text = {value["text"]}')

In [123]:
#Eksracting the disired data
#Dictionary of variables of the disired data 
var_pick = {'OMRÅDE': ['*'], 'BNØGLE': ['15110'], 'Tid': ['*']}

#Raw dataset 
bib_api = Dst.get_data(table_id = 'BIB1', variables = var_pick)
bib_api.sort_values(by= ['OMRÅDE', 'TID'])

bib_api.head(10)

,OMRÅDE,BNØGLE,TID,INDHOLD
0,All Denmark,Loan. All materials,2009,48370377
1,Region Nordjylland,Loan. All materials,2009,4772721
2,Region Midtjylland,Loan. All materials,2009,10657591
3,Region Syddanmark,Loan. All materials,2009,10164292
4,Region Hovedstaden,Loan. All materials,2009,15973419
5,Region Sjælland,Loan. All materials,2009,6802354
6,Copenhagen,Loan. All materials,2009,4515774
7,Frederiksberg,Loan. All materials,2009,957136
8,Ballerup,Loan. All materials,2009,536955
9,Brøndby,Loan. All materials,2009,209132


# Importing dataset 2: number of people in the ages (##)

In [ ]:
#Subjects within DST 
Dst.get_subjects()


In [124]:
#List of tables within the subject with index 2
tables = Dst.get_tables(subjects=['1'])
display(tables)

,id,text,unit,updated,firstPeriod,latestPeriod,active,variables
0,FOLK1A,Population at the first day of the quarter,Number,2023-02-10 08:00:00,2008Q1,2023Q1,True,"[region, sex, age, marital status, time]"
1,FOLK1AM,Population at the first day of the month,Number,2023-03-10 08:00:00,2021M10,2023M02,True,"[region, sex, age, time]"
2,BEFOLK1,Population 1. January,Number,2023-03-01 08:00:00,1971,2023,True,"[sex, age, marital status, time]"
3,BEFOLK2,Population 1. January,Number,2023-03-01 08:00:00,1901,2023,True,"[sex, age, time]"
4,FOLK3,Population 1. January,Number,2023-02-10 08:00:00,2008,2023,True,"[day of birth, birth month, year of birth, time]"
...,...,...,...,...,...,...,...,...
354,KM22,Registration and resignation from the national...,Number,2023-03-03 08:00:00,2007Q1,2022Q4,True,"[deanary, age, member of the National Church, ..."
355,KM33,Live births and deaths,Number,2023-03-03 08:00:00,2007Q1,2022Q4,True,"[deanary, movement, time]"
356,KMSTA004,Summary vital statistics,Number,2023-03-03 08:00:00,2015,2022,True,"[deanary, movements, time]"
357,KM4,Religious ceremonies,Number,2023-02-16 08:00:00,2006,2022,True,"[parish, movement, time]"


In [125]:
#tables = [tables.id == 'BEFOLK1']
#print(tables)

#list of variables (again?)
folk_vars = Dst.get_variables(table_id = 'FOLK1A')
folk_vars

for id in ['OMRÅDE', 'Tid']:
    print(id)
    values = folk_vars.loc[folk_vars.id == id, ['values']].values[0,0]
    for value in values:
        print(f'id= {value["id"]}, text = {value["text"]}')



OMRÅDE
id= 000, text = All Denmark
id= 084, text = Region Hovedstaden
id= 101, text = Copenhagen
id= 147, text = Frederiksberg
id= 155, text = Dragør
id= 185, text = Tårnby
id= 165, text = Albertslund
id= 151, text = Ballerup
id= 153, text = Brøndby
id= 157, text = Gentofte
id= 159, text = Gladsaxe
id= 161, text = Glostrup
id= 163, text = Herlev
id= 167, text = Hvidovre
id= 169, text = Høje-Taastrup
id= 183, text = Ishøj
id= 173, text = Lyngby-Taarbæk
id= 175, text = Rødovre
id= 187, text = Vallensbæk
id= 201, text = Allerød
id= 240, text = Egedal
id= 210, text = Fredensborg
id= 250, text = Frederikssund
id= 190, text = Furesø
id= 270, text = Gribskov
id= 260, text = Halsnæs
id= 217, text = Helsingør
id= 219, text = Hillerød
id= 223, text = Hørsholm
id= 230, text = Rudersdal
id= 400, text = Bornholm
id= 411, text = Christiansø
id= 085, text = Region Sjælland
id= 253, text = Greve
id= 259, text = Køge
id= 350, text = Lejre
id= 265, text = Roskilde
id= 269, text = Solrød
id= 320, text = 

In [126]:
#Loading the data 
var_pick = {'OMRÅDE': ['*'], 'Tid':['*']}
bef_api = Dst.get_data(table_id='FOLK1A', variables=var_pick)
bef_api.sort_values(by=['OMRÅDE'], inplace=True)

In [127]:
bef_api.head(10)

,OMRÅDE,TID,KØN,ALDER,CIVILSTAND,INDHOLD
4385,Aabenraa,2008Q3,Total,Total,Total,60427
823,Aabenraa,2020Q2,Total,Total,Total,58694
3767,Aabenraa,2011Q1,Total,Total,Total,59795
812,Aabenraa,2014Q4,Total,Total,Total,58917
1223,Aabenraa,2021Q2,Total,Total,Total,58587
4738,Aabenraa,2013Q3,Total,Total,Total,59197
3226,Aabenraa,2021Q4,Total,Total,Total,58595
808,Aabenraa,2010Q2,Total,Total,Total,59970
767,Aabenraa,2022Q2,Total,Total,Total,58770
5411,Aabenraa,2020Q3,Total,Total,Total,58735


In [129]:
bef_api_q1 = bef_api_q1.rename(columns={'TID': 'new_tid'})
bef_api_q1['new_tid'] = bef_api_q1['new_tid'].str.slice(stop=4)
bef_api_q1.rename(columns={'new_tid': 'TID'})

,OMRÅDE,TID,KØN,ALDER,CIVILSTAND,INDHOLD
3767,Aabenraa,2011,Total,Total,Total,59795
5469,Aabenraa,2012,Total,Total,Total,59600
2115,Aabenraa,2015,Total,Total,Total,58904
2092,Aabenraa,2017,Total,Total,Total,59003
1648,Aabenraa,2019,Total,Total,Total,59035
...,...,...,...,...,...,...
6199,Ærø,2012,Total,Total,Total,6636
3724,Ærø,2011,Total,Total,Total,6663
3730,Ærø,2016,Total,Total,Total,6290
1793,Ærø,2021,Total,Total,Total,5960


In [ ]:
bef_api_q1.head(10)
bib_api.head(10)


I = bib_api['BNØGLE'] == 'Loan. All materials'

bib_merge_test = bib_api.loc[I]
print(bib_merge_test)

#bib_merge_test = bib_api[['OMRÅDE', 'TID', 'BNØGLE']].loc['BNØGLE' == 'Loan. All materials']
#print(bib_merge_test)

# Merging the 2 datasets

In [130]:
#The datasets

print(f'Library dataset contains data from: {np.min(bib_api.TID.unique())} to {np.max(bib_api.TID)}')
print(f'Municipalities in Library data = {len(bib_api.OMRÅDE.unique())}')

print(f'Bef dataset contains data from: {np.min(bef_api_q1.new_tid.unique())} to {np.max(bef_api_q1.new_tid)}')
print(f'Municipalities in Library data = {len(bef_api_q1.OMRÅDE.unique())}')

#Differenses in the datasets
diff_mun = [m for m in bef_api_q1.OMRÅDE.unique() if m not in bib_api.OMRÅDE.unique()]

print(diff_mun)

Library dataset contains data from: 2009 to 2021
Municipalities in Library data = 103
Bef dataset contains data from: 2008 to 2023
Municipalities in Library data = 105
['Christiansø', 'Fanø']


In [ ]:
I = bib_merge_test['TID'] == '2019'


In [132]:
inner = pd.merge(bib_api, bef_api_q1, how='inner', on=['OMRÅDE', 'TID'])

KeyError: 'TID'

In [ ]:
bef_api_q1.rename(columns = {'new_tid': 'TID'})

In [ ]:
inner = pd.merge(bib_api, bef_api_q1, how= 'inner', on= ['OMRÅDE', 'TID'])

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.